## Brute force solver for Sudoku

Brute force is seldom the way to solve a puzzle for which there are well documented algorithms. However, brute force is the simplest way to solve *every* Sudoku puzzle.

When solving Sudoku puzzles algorthimicly, some puzzles can be solved with a single algorithm while others require multiple agorithms of varying degree of complexity.

In [1]:
from collections import Counter

### Helper functions

In [2]:
def is_cell_invalid(puz,pos):
    """Validate there are no duplicates at the specified position by checking that 
    position's row, column, and sector for duplciates."""

    def check_for_dupes(contents):
        """Check for duplicates in the specified array. This function does not know
        the difference between a row, column, or sector."""
        counter = Counter(contents)
        if counter[0] == 9:
            return True
        del counter[0]
        if counter.most_common(1)[0][1] > 1:
            return True
        else:
            return False

    # check this row
    row = pos // 9  # integer division determines the row
    if check_for_dupes(puz[9*row:9*row+9]):
        return True

    # check this column
    col = pos % 9   # remainder determines column
    column_contents = [puz[idy*9+col] for idy in range(9)]
    if check_for_dupes(column_contents):
        return True
    
    # check this section; determining the range of values that make up a 3x3 section
    # is more complex that determining the row or column
    section_start = (row - row % 3) * 9 + (col // 3) * 3
    section_contents = [puz[section_start+idc+idr*9] for idc in range(3) for idr in range(3)]
    if check_for_dupes(section_contents):
        return True
    
    return False


def validate_puzzle(puz):
    """Validate the entire puzzle has no errors by validating key cells."""
    for position in [0, 12, 24, 28, 40, 52, 56, 68, 80]:
        if is_cell_invalid(puz,position):
            return False  
    
    return True

In [3]:
def pprint_puzzle(puzzle):
    """Pretty print a single single Sudoku board with borders."""
    my_puz = puzzle.copy()

    for idx in range(81):
        my_puz[idx] = " " if my_puz[idx] == 0 else my_puz[idx]

    print(F'+{"-"*7}+{"-"*7}+{"-"*7}+')
    for ids in range(3):
        for idr in range(3):
            row = ids*3 + idr
            start = row * 9
            print(F'| {my_puz[start]} {my_puz[start+1]} {my_puz[start+2]} ', end='')
            print(F'| {my_puz[start+3]} {my_puz[start+4]} {my_puz[start+5]} ', end='')
            print(F'| {my_puz[start+6]} {my_puz[start+7]} {my_puz[start+8]} |')

        print(F'+{"-"*7}+{"-"*7}+{"-"*7}+')


def pprint_dual_puzzles(puzzle1,puzzle2):
    """Pretty print two Sudoku boards with border side by side."""
    arr1 = puzzle1.copy()
    arr2 = puzzle2.copy()
    for idx in range(81):
        arr1[idx] = " " if arr1[idx] == 0 else arr1[idx]
        arr2[idx] = " " if arr2[idx] == 0 else arr2[idx]

    print(F'+{"-"*7}+{"-"*7}+{"-"*7}+  +{"-"*7}+{"-"*7}+{"-"*7}+')
    for ids in range(3):
        for idr in range(3):
            row = ids*3 + idr
            start = row * 9
            print(F'| {arr1[start]} {arr1[start+1]} {arr1[start+2]} ', end='')
            print(F'| {arr1[start+3]} {arr1[start+4]} {arr1[start+5]} ', end='')
            print(F'| {arr1[start+6]} {arr1[start+7]} {arr1[start+8]} |', end='  ')
            print(F'| {arr2[start]} {arr2[start+1]} {arr2[start+2]} ', end='')
            print(F'| {arr2[start+3]} {arr2[start+4]} {arr2[start+5]} ', end='')
            print(F'| {arr2[start+6]} {arr2[start+7]} {arr2[start+8]} |')

        print(F'+{"-"*7}+{"-"*7}+{"-"*7}+  +{"-"*7}+{"-"*7}+{"-"*7}+')

### Brute force solver

In [4]:
def brute_force_solver(puzzle):

    history = []
    position = 0

    while 0 in puzzle:

        # find the next empty location without leaving the board
        while(position < 80 and puzzle[position] > 0):
            position += 1

        # the empty position is initialized to 1 before being checked by the
        # while loop that follows
        puzzle[position] = 1
        
        # keep incrementing the current location until the puzzle is valid;
        # the "current" location might be changed by the code inside the loop
        while(is_cell_invalid(puzzle,position)):

            # if the current value is 9 or greater, then we have run out of guesses
            # that could work and one of our previous guesses is actually invalid;
            # we must "erase" this current position and go back to the last position
            # that we set (which might also be a 9, hence the while loop).
            # this loop could take us back several guesses
            while puzzle[position] >= 9:
                puzzle[position] = 0
                position = history.pop()    # go back
            
            # increment the location by 1 and the while loop will check if its valid
            puzzle[position] += 1

        history.append(position)



In [5]:
# medium
puzzle = [int(x) for x in list('.29.71..3..8...6..3...5....5.....97......4...4.75.8..1.6.42.3..2..9....6.916...52'.replace('.','0'))]
# hard
puzzle = [int(x) for x in list('82.5.........3.257...67.9..4.61...3...........5..8419.9.2.1......57...2.......561'.replace('.','0'))]
# expert
puzzle = [int(x) for x in list('4......787..1..4.9..237....1..4.6....6........4..531.....5.....813.....7....2....'.replace('.','0'))]
# evil
# puzzle = [int(x) for x in list('9.......4.3..7.69...28......5......1....4...38..7..45.3....9..........1..9..6.57.'.replace('.','0'))]
# escargot
# puzzle = [int(x) for x in list('1....7.9..3..2...8..96..5....53..9...1..8...26....4...3......1..4......7..7...3..'.replace('.','0'))]

original = puzzle.copy()

brute_force_solver(puzzle)
pprint_dual_puzzles(original,puzzle)


+-------+-------+-------+  +-------+-------+-------+
| 4     |       |   7 8 |  | 4 5 1 | 9 6 2 | 3 7 8 |
| 7     | 1     | 4   9 |  | 7 3 6 | 1 8 5 | 4 2 9 |
|     2 | 3 7   |       |  | 9 8 2 | 3 7 4 | 5 6 1 |
+-------+-------+-------+  +-------+-------+-------+
| 1     | 4   6 |       |  | 1 7 5 | 4 9 6 | 8 3 2 |
|   6   |       |       |  | 3 6 9 | 2 1 8 | 7 4 5 |
|   4   |   5 3 | 1     |  | 2 4 8 | 7 5 3 | 1 9 6 |
+-------+-------+-------+  +-------+-------+-------+
|       | 5     |       |  | 6 2 7 | 5 3 1 | 9 8 4 |
| 8 1 3 |       |     7 |  | 8 1 3 | 6 4 9 | 2 5 7 |
|       |   2   |       |  | 5 9 4 | 8 2 7 | 6 1 3 |
+-------+-------+-------+  +-------+-------+-------+


In [6]:
validate_puzzle(puzzle)

True